##  Explore and cluster the neighborhoods in Toronto

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [107]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup

The following read the Wikipedia page to text.

In [108]:
url = " https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = rq.get(url)

Use beautifulsoup to extract table.

In [109]:
soup = BeautifulSoup(data.content, 'html.parser')
table = soup.find('tbody')
lines = table.select('tr')
line = [l.get_text() for l in lines]
#print(line[:5])

Create pandas DataFrame:

In [110]:
df = pd.DataFrame(line)
df = df[0].str.split('\n', expand = True)
df = df.rename(columns = df.iloc[0])
df = df.drop(df.index[0])
df.head()

,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,


Ignore cells with a borough that is Not assigned.

In [111]:
df = df[df.Borough !='Not assigned']
df.head()

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,


Combined neighbourhoods with same postcode:

In [112]:
df1 = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df1.reset_index(inplace = True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


If a cell has a borough but a Not assigned neighborhood, then set the neighborhood the same as the borough:

In [113]:
df1 = df1.replace("Not assigned", df.Borough)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Get the shape of the DataFrame:

In [114]:
df1.shape

(103, 3)

Get Latitude and Longitude of Toronto from the provided csv link:

In [115]:
pc = pd.read_csv('http://cocl.us/Geospatial_data')
pc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [116]:
df2 = pd.merge(df1, pc, left_on=['Postcode'], right_on=['Postal Code'], how='inner') 
df2 = df2.drop(['Postal Code'], axis=1)
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


Create a map of Toronto with neighborhoods superimposed on top.

In [117]:
import folium
from geopy.geocoders import Nominatim
# create map of Toronto using latitude and longitude values
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto